
This notebook conducts a comprehensive evaluation of our tuned models on the full dataset, selects the best model for deployment, and generates detailed performance reports and visualizations.

## Table of Contents
1. Model Loading and Setup
2. Final Training on Full Dataset
3. Comprehensive Model Evaluation
4. Performance Visualization
5. Model Selection and Justification
6. Model Persistence and Reporting

## Environment Setup and Imports

In [8]:
# Standard data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Model persistence
from joblib import dump, load

# Evaluation metrics and tools
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    make_scorer
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# Base models
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor
)

# Set random seed for reproducibility
np.random.seed(42)

# Configure visualizations
plt.style.use('default')  # Use default matplotlib style
sns.set_theme()  # Set seaborn defaults
plt.rcParams['figure.figsize'] = [12, 8]

# Create directories for outputs
figures_dir = Path('figures')
models_dir = Path('models')
results_dir = Path('results')

for dir_path in [figures_dir, models_dir, results_dir]:
    dir_path.mkdir(exist_ok=True)


Load the full preprocessed dataset and the tuned models from our previous analysis.

In [9]:
data = pd.read_csv('../../data/processed/engineered_features.csv')

# Selected features from previous analysis
selected_features = [
    'risk_score_rolling_std',  # Primary predictors
    'dist_from_center',
    'time_window',
    'y',
    'hour',
    'offender_target_ratio',   # Secondary predictors
    'grid_id',
    'num_nearby',
    'dist_to_high_risk',       # Supporting predictors
    'day_of_week',
    'event_rate'
]

# Prepare features and target
X = data[selected_features]
y = data['risk_score']

# Clean the data by replacing infinite values with NaN and then filling them
X = X.replace([np.inf, -np.inf], np.nan)

# For each column, fill NaN with the median of that column
for column in X.columns:
    X[column] = X[column].fillna(X[column].median())

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create preprocessor
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numeric_features)
    ]
)

print("Dataset Overview:")
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print("\nFeature Summary:")
print(X.describe())

Dataset Overview:
Training set shape: (320000, 11)
Test set shape: (80000, 11)

Feature Summary:
       risk_score_rolling_std  dist_from_center   time_window              y  \
count           400000.000000     400000.000000  400000.00000  400000.000000   
mean                 0.044015         38.928369      82.83400      50.135355   
std                  0.029310         14.302932      48.11287      29.287526   
min                  0.000000          0.000000       0.00000       0.000000   
25%                  0.022878         29.068884      41.00000      25.000000   
50%                  0.038877         40.447497      83.00000      50.000000   
75%                  0.059539         49.497475     124.25000      76.000000   
max                  0.308365         70.710678     166.00000      99.000000   

               hour  offender_target_ratio        grid_id     num_nearby  \
count  400000.00000          400000.000000  400000.000000  400000.000000   
mean       11.43600           


Create models with the best hyperparameters found during tuning.

In [10]:
rf_best_params = {
    'n_estimators': 150,
    'max_depth': 20,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt'
}

gb_best_params = {
    'n_estimators': 100,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.8,
    'max_features': 'sqrt'
}

# Initialize models with best parameters
rf_model = RandomForestRegressor(**rf_best_params, random_state=42)
gb_model = GradientBoostingRegressor(**gb_best_params, random_state=42)
baseline_model = LinearRegression()

# Create pipelines
def create_pipeline(model):
    return Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

rf_pipeline = create_pipeline(rf_model)
gb_pipeline = create_pipeline(gb_model)
baseline_pipeline = create_pipeline(baseline_model)

# Create voting ensemble
voting_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', VotingRegressor([
        ('rf', rf_model),
        ('gb', gb_model)
    ]))
])


Define helper functions for model evaluation and visualization.

In [17]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Comprehensive model evaluation function."""
    # Train the model
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train)
    
    # Make predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # Calculate metrics
    metrics = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, train_pred)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, test_pred)),
        'train_mae': mean_absolute_error(y_train, train_pred),
        'test_mae': mean_absolute_error(y_test, test_pred),
        'train_r2': r2_score(y_train, train_pred),
        'test_r2': r2_score(y_test, test_pred)
    }
    
    print(f"\n{model_name} Performance:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    return metrics, train_pred, test_pred

def plot_feature_importance(model, feature_names, title="Feature Importance"):
    """Plot feature importance for tree-based models."""
    if hasattr(model.named_steps['regressor'], 'feature_importances_'):
        importances = model.named_steps['regressor'].feature_importances_
        indices = np.argsort(importances)[::-1]  # Sort in descending order
        
        plt.figure(figsize=(10, 6))
        plt.barh(range(len(indices)), importances[indices])
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
        plt.xlabel('Relative Importance')
        plt.title(title)
        plt.tight_layout()
        return plt.gcf()

def plot_residuals(y_true, y_pred, title="Residual Plot"):
    """Create residual plot with distribution."""
    residuals = y_true - y_pred
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Scatter plot of residuals
    ax1.scatter(y_pred, residuals, alpha=0.5)
    ax1.axhline(y=0, color='r', linestyle='--')
    ax1.set_xlabel('Predicted Values')
    ax1.set_ylabel('Residuals')
    ax1.set_title(f'{title} - Scatter')
    
    # Distribution of residuals
    sns.histplot(residuals, kde=True, ax=ax2)
    ax2.axvline(x=0, color='r', linestyle='--')
    ax2.set_xlabel('Residual Value')
    ax2.set_title(f'{title} - Distribution')
    
    plt.tight_layout()
    return fig

def temporal_performance(model, X, y, time_column):
    """Calculate performance metrics across time periods."""
    predictions = model.predict(X)
    time_periods = X[time_column].unique()
    temporal_metrics = []
    
    for period in sorted(time_periods):
        mask = X[time_column] == period
        period_pred = predictions[mask]
        period_true = y[mask]
        
        metrics = {
            'period': period,
            'rmse': np.sqrt(mean_squared_error(period_true, period_pred)),
            'r2': r2_score(period_true, period_pred)
        }
        temporal_metrics.append(metrics)
    
    return pd.DataFrame(temporal_metrics)

def calculate_prediction_intervals(model, X, percentile=95):
    """Calculate prediction intervals using bootstrapping for ensemble models.
    
    Args:
        model: Fitted pipeline with an ensemble model
        X: Input features
        percentile: The percentile range for the prediction interval (default: 95)
    
    Returns:
        lower, upper: Lower and upper bounds of the prediction interval
    """
    # Get predictions from all trees in the forest
    predictions = []
    forest = model.named_steps['regressor']
    
    # Apply preprocessing to input data
    X_processed = model.named_steps['preprocessor'].transform(X)
    
    # Get predictions from each tree
    for estimator in forest.estimators_:
        pred = estimator.predict(X_processed)
        predictions.append(pred)
    
    # Convert to numpy array for percentile calculation
    predictions = np.array(predictions)
    
    # Calculate prediction intervals
    lower = np.percentile(predictions, (100 - percentile) / 2, axis=0)
    upper = np.percentile(predictions, 100 - (100 - percentile) / 2, axis=0)
    
    return lower, upper


Train and evaluate all models on the full dataset.

In [12]:
models = {
    'Baseline (Linear)': baseline_pipeline,
    'Random Forest': rf_pipeline,
    'Gradient Boosting': gb_pipeline,
    'Voting Ensemble': voting_pipeline
}

results = {}
predictions = {}

for name, model in models.items():
    metrics, train_pred, test_pred = evaluate_model(
        model, X_train, X_test, y_train, y_test, name
    )
    results[name] = metrics
    predictions[name] = {'train': train_pred, 'test': test_pred}

# Convert results to DataFrame for easy comparison
results_df = pd.DataFrame(results).round(4)
print("\nModel Comparison:")
print(results_df)


Training Baseline (Linear)...

Baseline (Linear) Performance:
train_rmse: 0.1204
test_rmse: 0.1206
train_mae: 0.0851
test_mae: 0.0853
train_r2: 0.1413
test_r2: 0.1412

Training Random Forest...

Baseline (Linear) Performance:
train_rmse: 0.1204
test_rmse: 0.1206
train_mae: 0.0851
test_mae: 0.0853
train_r2: 0.1413
test_r2: 0.1412

Training Random Forest...

Random Forest Performance:
train_rmse: 0.0676
test_rmse: 0.0907
train_mae: 0.0483
test_mae: 0.0636
train_r2: 0.7290
test_r2: 0.5141

Training Gradient Boosting...

Random Forest Performance:
train_rmse: 0.0676
test_rmse: 0.0907
train_mae: 0.0483
test_mae: 0.0636
train_r2: 0.7290
test_r2: 0.5141

Training Gradient Boosting...

Gradient Boosting Performance:
train_rmse: 0.1151
test_rmse: 0.1156
train_mae: 0.0819
test_mae: 0.0823
train_r2: 0.2148
test_r2: 0.2108

Training Voting Ensemble...

Gradient Boosting Performance:
train_rmse: 0.1151
test_rmse: 0.1156
train_mae: 0.0819
test_mae: 0.0823
train_r2: 0.2148
test_r2: 0.2108

Training 


Create comprehensive visualizations of model performance.

In [18]:
plt.figure(figsize=(12, 8))
for name, preds in predictions.items():
    plt.scatter(y_test, preds['test'], alpha=0.5, label=name)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel('Actual Risk Score')
plt.ylabel('Predicted Risk Score')
plt.title('Predicted vs Actual - All Models')
plt.legend()
plt.savefig(figures_dir / 'predicted_vs_actual.png')
plt.close()

# 2. Feature Importance Plots
for name, model in models.items():
    if hasattr(model.named_steps['regressor'], 'feature_importances_'):
        fig = plot_feature_importance(model, selected_features, f"{name} - Feature Importance")
        fig.savefig(figures_dir / f'feature_importance_{name.lower().replace(" ", "_")}.png')
        plt.close()

# 3. Residual Plots
for name, preds in predictions.items():
    fig = plot_residuals(y_test, preds['test'], f"{name} - Residuals")
    fig.savefig(figures_dir / f'residuals_{name.lower().replace(" ", "_")}.png')
    plt.close()

# 4. Temporal Performance
temporal_results = {}
for name, model in models.items():
    temporal_metrics = temporal_performance(model, X_test, y_test, 'time_window')
    temporal_results[name] = temporal_metrics

# Plot temporal performance
plt.figure(figsize=(12, 6))
for name, metrics in temporal_results.items():
    plt.plot(metrics['period'], metrics['r2'], label=name, marker='o')
plt.xlabel('Time Window')
plt.ylabel('R² Score')
plt.title('Model Performance Over Time')
plt.legend()
plt.savefig(figures_dir / 'temporal_performance.png')
plt.close()


Based on the comprehensive evaluation, select the best model for deployment.

In [19]:
best_model = rf_pipeline
best_model_name = 'Random Forest'

# Calculate prediction intervals for the best model
if isinstance(best_model.named_steps['regressor'], RandomForestRegressor):
    lower, upper = calculate_prediction_intervals(best_model, X_test)
    prediction_intervals = pd.DataFrame({
        'actual': y_test,
        'predicted': predictions[best_model_name]['test'],
        'lower_bound': lower,
        'upper_bound': upper
    })
    
    # Plot predictions with intervals
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, predictions[best_model_name]['test'], alpha=0.5)
    plt.fill_between(y_test, lower, upper, alpha=0.2, label='95% Prediction Interval')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
    plt.xlabel('Actual Risk Score')
    plt.ylabel('Predicted Risk Score')
    plt.title(f'{best_model_name} Predictions with 95% Intervals')
    plt.legend()
    plt.savefig(figures_dir / 'best_model_predictions_with_intervals.png')
    plt.close()

# Save results
results_df.to_csv(results_dir / 'final_model_results.csv')
prediction_intervals.to_csv(results_dir / 'prediction_intervals.csv')

# Save best model
dump(best_model, models_dir / 'best_model.joblib')

['models\\best_model.joblib']